# Environnements Python : venv vs conda vs uv

Ce notebook présente les options d’environnement Python les plus courantes, comment les créer, les activer, installer des dépendances, les désactiver, et comment vérifier que tout fonctionne.

## Pré-requis
- Avoir **Python** installé (pour `venv`) et/ou **Conda** (miniconda) installé (pour `conda`), et/ou **uv** installé (pour `uv`).
- Ouvrir un terminal dans le dossier du projet (ou utiliser un terminal intégré à l’IDE).

## Principe
Un *environnement* isole :
- la version de Python,
- les bibliothèques installées,
- les dépendances d’un projet.

Cela évite les conflits entre projets et rend le travail reproductible.


## 0) Installation des outils (si nécessaire)

> Cette section est à exécuter **une seule fois sur votre machine**.
> Les commandes suivantes sont à lancer dans un **terminal**, pas dans une cellule Python.

### Installer Conda (Miniconda recommandé)

Conda n’est pas inclus avec Python.
La version légère recommandée est **Miniconda**.

- **macOS / Linux**

```bash
# Télécharger le script d'installation (exemple pour macOS ARM)
curl -O https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-arm64.sh

# Lancer l'installation
bash Miniconda3-latest-MacOSX-arm64.sh
```

Puis redémarrer le terminal.

Vérification :

```bash
conda --version
```

- **Windows**

1. Télécharger l’installateur depuis :
   [https://docs.conda.io/en/latest/miniconda.html](https://docs.conda.io/en/latest/miniconda.html)

2. Lancer l’installation (cocher "Add to PATH" si proposé).

Vérification dans PowerShell :

```powershell
conda --version
```

### Installer uv

`uv` est un outil moderne écrit en Rust.
Il est indépendant de Conda.

- **macOS / Linux**

```bash
curl -LsSf https://astral.sh/uv/install.sh | sh
```

Puis redémarrer le terminal.

Vérification :

```bash
uv --version
```

- **Windows (PowerShell)**

```powershell
powershell -c "irm https://astral.sh/uv/install.ps1 | iex"
```

Puis rouvrir PowerShell.

Vérification :

```powershell
uv --version
```

### Option alternative : installer uv via pip

Si Python est déjà installé :

```bash
pip install uv
```

Puis vérifier :

```bash
uv --version
```

### Remarque importante

* `venv` est inclus dans Python.
* `conda` est un gestionnaire d’environnements **et** de paquets.
* `uv` est un gestionnaire moderne rapide compatible avec l’écosystème `pip`.

Dans un contexte professionnel moderne orienté ML/Data :

* `venv` → solution simple et universelle
* `conda` → utile pour stacks scientifiques lourdes
* `uv` → recommandé pour projets modernes reproductibles

## 1) Option standard : `venv` + `pip`

### Création (dans le dossier du projet)
```bash
python -m venv .venv
```

### Activation

- **macOS / Linux**

```bash
source .venv/bin/activate
```

- **Windows (PowerShell)**

```powershell
.venv\Scripts\Activate.ps1
```

### Installer des dépendances

```bash
python -m pip install -U pip
pip install numpy pandas scikit-learn matplotlib
```

### Sauvegarder les dépendances dans un fichier requirements.txt (approche historique)
```bash
pip freeze > requirements.txt
```

### Réinstaller
```bash
pip install -r requirements.txt
```

### Désactiver l’environnement
```bash
deactivate
```


## 2) Option Conda : `conda create` + `conda install`

Conda est particulièrement utile quand vous avez des dépendances lourdes (scientifiques, binaires), ou quand vous voulez standardiser une stack data/ML.

### Création
```sh
conda create -n demo-env python=3.11 -y
```

### Activation
```bash
conda activate demo-env
```

### Installer des dépendances

**Avec conda (recommandé pour les bibliothèques avec des dépendances binaires) :**
  
```bash
conda install -y numpy pandas scikit-learn matplotlib
```
  
**Avec pip (ce que je recommande pour les bibliothèques purement Python):**
  
```bash
pip install numpy pandas scikit-learn matplotlib
```
  
### Export (reproductible)
```bash
conda env export > environment.yml
```

### Recréer depuis le fichier
```bash
conda env create -f environment.yml
conda activate demo-env
```

### Désactiver l’environnement
```bash
conda deactivate
```


## 3) Option moderne : `uv` (rapide + reproductible)

`uv` gère :
- la création d’un `venv`,
- l’installation rapide des paquets,
- et peut fonctionner en mode *projet* avec `pyproject.toml` + lock.

### A) Mode simple (équivalent `venv` + `pip`)

Créer un environnement local `.venv` :
```bash
uv venv
```

Avec une version spécifique de Python :
```bash
uv venv --python 3.11
```

Activer (optionnel) :
- **macOS / Linux**
```bash
source .venv/bin/activate
```
- **Windows (PowerShell)**
```powershell
.venv\Scripts\Activate.ps1
```

Installer des dépendances :
```bash
uv pip install numpy pandas scikit-learn matplotlib
```

Désactiver (si vous avez activé) :
```bash
deactivate
```

### B) Mode projet (recommandé)

Initialiser un projet et enregistrer les dépendances dans `pyproject.toml` :
```bash
uv init
uv add numpy pandas scikit-learn matplotlib
uv sync
```

Lancer un script **sans activer** l’environnement (pratique et reproductible) :
```bash
uv run python demo_env_test.py
```

Dans ce cas, il n’y a rien à désactiver : `uv run ...` utilise automatiquement l’environnement du projet.


## 4) Vérification : quel Python et quelles versions ?

Le code suivant permet de :
- afficher le chemin de l’interpréteur (`sys.executable`),
- afficher la version de Python,
- afficher les versions de quelques librairies,
- exécuter une mini démo ML (régression linéaire) pour valider l’installation.

Vous pouvez l’exécuter **dans un notebook** ou le sauvegarder en **script `demo_env_test.py`**.


In [2]:
from __future__ import annotations

import platform
import sys

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


def run_environment_sanity_check(seed: int = 42) -> None:
    """Print environment information and run a small ML demo.

    This function is intentionally small and deterministic so it can be used
    as a quick check that the active environment is the one expected.
    """

    print("=== Environment sanity check ===")
    print(f"Python executable: {sys.executable}")
    print(f"Python version:    {sys.version.split()[0]}")
    print(f"Platform:          {platform.platform()}")

    print("\n=== Package versions ===")
    print(f"numpy:  {np.__version__}")
    print(f"pandas: {pd.__version__}")

    # Synthetic dataset (simple linear relationship + noise)
    rng = np.random.default_rng(seed)
    X = rng.normal(size=(200, 3))
    y = 3.0 * X[:, 0] - 2.0 * X[:, 1] + 0.5 * X[:, 2] + rng.normal(scale=0.2, size=200)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=seed
    )

    model = LinearRegression()
    model.fit(X_train, y_train)

    r2 = model.score(X_test, y_test)
    print("\n=== Quick ML demo ===")
    print(f"LinearRegression R^2 on test: {r2:.4f}")
    print(f"Coefficients: {model.coef_.round(3)}")
    print(f"Intercept:    {model.intercept_:.3f}")


run_environment_sanity_check()


=== Environment sanity check ===
Python executable: /Users/jika/Desktop/ENSEIGMEMENT/Machine Learing/ALYRA - IA/ai-dev-teaching-hub/.venv/bin/python
Python version:    3.11.14
Platform:          macOS-26.2-arm64-arm-64bit

=== Package versions ===
numpy:  2.4.2
pandas: 3.0.0

=== Quick ML demo ===
LinearRegression R^2 on test: 0.9956
Coefficients: [ 3.005 -2.008  0.477]
Intercept:    -0.006


## Résumé : quand choisir quoi ?

- **venv + pip** : standard Python, simple, parfait pour la majorité des projets.
  
- **conda** : utile quand vous avez des dépendances scientifiques lourdes ou un écosystème conda déjà en place.
  
- **uv** : très rapide, bon support du mode *projet* (`pyproject.toml` + lock), excellent pour des projets modernes et reproductibles.
